In [1]:
import sympy as sp

# 1.

# 2. Consider the parametrization for the Torus $T$ given by:

$$\vec{X} = \left( (a + rcos(u))cos(v),\;\; (a + rcos(u))sin(v),\;\; rsin(u)  \right)$$

$$U = \left\{ (\theta, \phi) \in R^{2} | 0 < u < 2\pi,\;\; 0 < v < 2\pi  \right\}$$

with $0 < r < a$

- Is $T$ a regular surface? Justify your answer completely

In [2]:
u, v, a, r = sp.symbols('u v a r', real = True)
ulimits = (0, 2*sp.pi)
vlimits = (0, 2*sp.pi)

x = sp.Matrix([(a + r*sp.cos(u))*sp.cos(v), (a + r*sp.cos(u))*sp.sin(v), r*sp.sin(u)])
x

Matrix([
[(a + r*cos(u))*cos(v)],
[(a + r*cos(u))*sin(v)],
[             r*sin(u)]])

- Evaluate the tangent vectors on T

In [3]:
# Calculate the tangent vectors to the surface
xu = x.diff(u)
xu

Matrix([
[-r*sin(u)*cos(v)],
[-r*sin(u)*sin(v)],
[        r*cos(u)]])

In [4]:
xv = x.diff(v)
xv

Matrix([
[-(a + r*cos(u))*sin(v)],
[ (a + r*cos(u))*cos(v)],
[                     0]])

- Evaluate the coeficients of the first fundamental form

In [5]:
# Evaluate the coefficients of the first fundamental form
E = xu.dot(xu).simplify()
E

r**2

In [6]:
F = xu.dot(xv)
F

0

In [7]:
G = xv.dot(xv).simplify()
G

(a + r*cos(u))**2

- Which is the metric tensor for the Torus?

In [8]:
# Calculate the metric tensors for the surface
g = sp.Matrix([[E, F], [F, G]])
g

Matrix([
[r**2,                 0],
[   0, (a + r*cos(u))**2]])

- Which is the area element for the surface?

In [9]:
# Area element for the surface
ds = sp.sqrt(E*G - F**2)
ds

Abs(r)*Abs(a + r*cos(u))

- Evaluate the area of the surface.

In [17]:
# calculate the integral
I = sp.integrate(ds, (v, *vlimits))
I

Piecewise((2*pi*r*(a + r*cos(u)), (r >= 0) & (a + r*cos(u) >= 0)), (2*pi*r*(-a - r*cos(u)), r >= 0), (-2*pi*r*(a + r*cos(u)), a + r*cos(u) >= 0), (-2*pi*r*(-a - r*cos(u)), True))

- Evaluate the second fundamental form of the surface

In [20]:
# Second fundamental form of the surface
xuu = xu.diff(u)
xuv = xu.diff(v)
xvv = xv.diff(v)

In [21]:
L = xuu.dot(x)
M = xuv.dot(x)
N = xvv.dot(x)

- Evaluate and plot the principal curvatures of the surface

- Evaluate and plot Gaussian curvature of the surface

# 3. Consider the parametrization for the Möbius strip

$$\displaystyle \vec{x}(\theta, t) = \left(   \left(1 + \frac{1}{2}tcos(\frac{\theta}{2}) \right) cos(\theta),\;\; \left(1 + \frac{1}{2}tcos(\frac{\theta}{2}) \right) sin(\theta),\;\; \frac{1}{2}tsin(\frac{\theta}{2})  \right)$$

for 

$$\displaystyle U = \left\{ (\theta, t) \in R^{2} | -0.25 < t < 0.25,\;\; 0 < \theta < 2\pi  \right\}$$

- Evaluate the first fundamental form of the surface

- Evaluate the metric tensor for the mobius strip

- Which is the area element $\| \vec{X}_{\theta} \times \vec{X}_{t} \|$ for the surface?

- Evaluate the total area for the ellipsoid as 

$$\int \int_{Q} \| \vec{X}_{\theta} \times \vec{X}_{t} \| d\theta dt$$

If the analytic solution is not possible, use numerical integration.

- Obtain the normal vector as a function of the coordinates $(\theta, t)$

$$\vec{N}(\theta, t) = \frac{\vec{X}_{\theta} \times \vec{X}_{t}}{\| \vec{X}_{\theta} \times \vec{X}_{t} \|}$$

- Evaluate the normal vector at $t = 0$ and obtain the limits 

$$\lim_{\theta \to 0} \vec{N}(\theta, 0)$$

and 

$$\lim_{\theta \to 2\pi} \vec{N}(\theta, 0)$$